<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0f31926fe962d9d127c41728301da865c8ee0743addd8ba12af86d333f68265f
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-06 15:16:23
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 16.92 K (0.12%)
Current PnL: -24.77 L (-15.8%)
CY Booked + Current PnL: -10.48 L (-6.68%)
-------------------
Total profit:  2.06 L
Total loss:  -26.83 L
-------------------
Total Booked + Current PnL: 16.50 L (12.71%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.9%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.74 L (62.19%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.56,16.18,3.02,19.69,6728.0,31023.0,222765.0,4.86,70.0,X-MC,3.59,78.0,4.61,1.54,24.08,XY25,NTT,AC
51,NESTLEIND,1377.00,0.36,16.31,4.37,21.40,13491.0,43290.0,308716.0,7.98,71.0,X-LC,5.01,12.0,3.21,2.14,23.67,XY25,NTT,FMCG
49,MASFIN,398.61,5.37,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.61,164.0,0.18,0.70,47.28,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.42,-19.79,24.79,0.09,20039.0,-19943.0,80834.0,87.62,42.0,M-SC,3.28,253.0,-1.00,0.56,6.20,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.16,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,81.0,L-LC,5.65,179.0,0.35,1.09,27.27,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,12.92,2.98,44.33,48.63,91280.0,5957.0,205911.0,-31.50,69.0,H-SC,14.21,136.0,0.07,1.43,14.95,XR,NTT,MISC
56,RAJOOENG,143.10,11.14,-24.54,85.00,39.61,65748.0,-25150.0,77350.0,-48.92,69.0,H-SC,32.19,135.0,-0.38,0.54,25.20,AR,ATH,MISC
49,MASFIN,398.61,5.37,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.61,164.0,0.18,0.70,47.28,XR,ATH,FINANCE
0,5PAISA,593.00,3.81,-25.77,52.09,12.90,84489.0,-56298.0,162198.0,139.18,73.0,H-SC,10.96,161.0,-0.67,1.12,34.87,OX40N,NTT,FINANCE
31,HAVELLS,2069.16,3.31,-3.99,37.99,32.48,119629.0,-13096.0,314895.0,-8.81,71.0,X-MC,6.06,67.0,-0.11,2.18,7.89,X40,ATH,ELECTRICAL


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1952.00,-4.60,-3.36,29.66,25.30,57603.0,-6753.0,194210.0,-6.52,38.0,X-LC,8.96,17.0,-0.12,1.35,30.09,XY25,BTT,REFINERIES
83,WHIRLPOOL,2270.00,-4.10,-29.63,158.53,81.92,128062.0,-34017.0,80781.0,-58.80,31.0,M-SC,24.75,236.0,-0.27,0.56,0.00,XR,NTT,DURABLES
74,TITAGARH,1548.00,-2.76,-19.13,80.21,45.74,157093.0,-46325.0,195852.0,7.85,53.0,H-SC,2.98,174.0,-0.29,1.36,27.38,XY24,NTT,ENGINEERING
22,DIXON,18931.72,-2.63,-18.64,61.34,31.26,115162.0,-43024.0,187744.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
50,MEDANTA,1486.00,-2.34,0.92,22.95,24.08,29402.0,1162.0,128112.0,-7.25,54.0,X-SC,7.03,89.0,0.04,0.89,20.71,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.04,-0.92,109.85,107.91,160690.0,-1363.0,146281.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.07,OX40N,ATH,CABLES


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.04,-0.92,109.85,107.91,160690.0,-1363.0,146281.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.07,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.60,-15.33,18.13,0.02,26795.0,-26763.0,147796.0,125.45,52.0,M-SC,7.24,234.0,-1.00,1.02,28.56,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.42,-19.79,24.79,0.09,20039.0,-19943.0,80834.0,87.62,42.0,M-SC,3.28,253.0,-1.00,0.56,6.20,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.24,-19.93,41.73,13.47,90099.0,-53757.0,215910.0,-67.15,59.0,H-SC,5.30,158.0,-0.60,1.50,9.78,XY24,NTT,PAINTS
66,SIS,528.00,-0.09,-22.37,56.21,21.26,48447.0,-24842.0,86190.0,2035.75,57.0,H-SC,4.28,166.0,-0.51,0.60,16.45,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.52,3.68,16.45,20.74,34652.0,7480.0,210650.0,35.83,74.0,M-MC,6.85,189.0,0.22,1.46,14.69,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.16,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,81.0,L-LC,5.65,179.0,0.35,1.09,27.27,XY25,ATH,MINING
84,WIPRO,420.00,0.89,8.96,58.10,72.27,107268.0,15179.0,184627.0,-6.53,57.0,M-LC,5.82,99.0,0.14,1.28,15.69,XR,NTT,IT
49,MASFIN,398.61,5.37,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.61,164.0,0.18,0.70,47.28,XR,ATH,FINANCE
37,IEX,219.00,12.92,2.98,44.33,48.63,91280.0,5957.0,205911.0,-31.50,69.0,H-SC,14.21,136.0,0.07,1.43,14.95,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,5.37,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.61,164.0,0.18,0.70,47.28,XR,ATH,FINANCE
84,WIPRO,420.00,0.89,8.96,58.10,72.27,107268.0,15179.0,184627.0,-6.53,57.0,M-LC,5.82,99.0,0.14,1.28,15.69,XR,NTT,IT
37,IEX,219.00,12.92,2.98,44.33,48.63,91280.0,5957.0,205911.0,-31.50,69.0,H-SC,14.21,136.0,0.07,1.43,14.95,XR,NTT,MISC
25,FINCABLES,1641.55,0.04,-0.92,109.85,107.91,160690.0,-1363.0,146281.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.07,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,1.05,-1.76,38.06,35.63,78359.0,-3690.0,205884.0,-15.41,55.0,H-MC,2.95,119.0,-0.05,1.43,14.62,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-2.17,-16.15,32.13,10.78,64741.0,-38821.0,201497.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-2.63,-18.64,61.34,31.26,115162.0,-43024.0,187744.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
23,DMART,5391.45,0.46,-11.95,47.19,29.60,103714.0,-29820.0,219780.0,-24.94,30.0,X-LC,9.69,19.0,-0.29,1.52,7.61,X40N,ATH,FMCG
7,AWL,485.00,-1.22,-25.15,110.01,57.19,268539.0,-82023.0,244104.0,-64.65,30.0,X-MC,8.11,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.59,-10.55,46.02,30.62,97581.0,-25014.0,212040.0,-36.47,33.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,0.04,1.09,19.47,20.77,46083.0,2545.0,236689.0,-7.76,39.0,X-MC,0.95,62.0,0.06,1.64,8.08,X40N,NTT,BREWERIES
65,SIEMENS,4671.5,1.31,-15.99,49.40,25.51,77234.0,-29750.0,156345.0,-4.14,50.0,H-LC,1.34,49.0,-0.39,1.08,15.21,AR,ATH,ELECTRICAL
20,DABUR,735.0,-0.24,1.66,41.36,43.70,104300.0,4108.0,252176.0,-6.68,65.0,X-MC,1.95,72.0,0.04,1.75,16.96,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-0.47,-6.28,29.25,21.14,62323.0,-14269.0,213070.0,-12.86,40.0,X-MC,2.28,74.0,-0.23,1.48,18.33,XY24,NTT,PAINTS
45,JIOFIN,387.0,-0.75,-4.10,29.80,24.48,76410.0,-10956.0,256409.0,-5.71,49.0,X-LC,2.29,36.0,-0.14,1.78,48.62,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.22,-25.15,110.01,57.19,268539.0,-82023.0,244104.0,-64.65,30.0,X-MC,8.11,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-2.63,-18.64,61.34,31.26,115162.0,-43024.0,187744.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
52,PAGEIND,51605.08,-0.59,-10.55,46.02,30.62,97581.0,-25014.0,212040.0,-36.47,33.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS
43,ITC,452.00,-2.17,-16.15,32.13,10.78,64741.0,-38821.0,201497.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
11,BATAINDIA,2096.00,-1.07,-38.92,121.35,35.20,95373.0,-50077.0,78593.0,3.10,40.0,X-SC,17.24,92.0,-0.53,0.54,0.94,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,2.50,-26.44,94.17,42.84,45033.0,-17185.0,47821.0,-52.07,57.0,X-SC,29.20,83.0,-0.38,0.33,8.19,XY24,NTT,MISC
58,RELAXO,1176.00,-0.49,-47.64,190.91,52.31,145090.0,-69161.0,75999.0,-44.25,46.0,X-SC,7.38,91.0,-0.48,0.53,1.73,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.07,-38.92,121.35,35.20,95373.0,-50077.0,78593.0,3.10,40.0,X-SC,17.24,92.0,-0.53,0.54,0.94,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.34,0.92,22.95,24.08,29402.0,1162.0,128112.0,-7.25,54.0,X-SC,7.03,89.0,0.04,0.89,20.71,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.26,-15.91,70.44,43.32,96475.0,-25912.0,136960.0,-27.69,54.0,X-SC,7.76,90.0,-0.27,0.95,5.16,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.19,-10.51,34.89,20.71,118091.0,-39769.0,338468.0,-22.98,55.0,X-LC,2.66,1.0,-0.34,2.35,13.09,X40,ATH,IT
41,INFY,1972.00,0.36,9.75,22.32,34.24,76647.0,30511.0,343399.0,-14.57,50.0,X-LC,7.46,2.0,0.40,2.38,18.84,X40,NTT,IT
75,TMPV,600.00,-1.27,-24.41,62.69,22.98,107046.0,-55144.0,170754.0,-21.86,57.0,X-LC,4.96,3.0,-0.52,1.18,7.40,XY24,NTT,AUTO
81,VBL,671.64,2.01,0.70,34.73,35.67,110456.0,2201.0,318043.0,-11.24,65.0,X-LC,2.36,4.0,0.02,2.20,14.65,X40N,ATH,FMCG
43,ITC,452.00,-2.17,-16.15,32.13,10.78,64741.0,-38821.0,201497.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.56,-32.52,96.94,32.89,50504.0,-25108.0,52098.0,-743.10,73.0,L-MC,7.19,259.0,-0.50,0.36,43.47,XR,NTT,BANKS
49,MASFIN,398.61,5.37,3.41,18.02,22.05,18259.0,3345.0,101325.0,-11.02,64.0,H-SC,6.61,164.0,0.18,0.70,47.28,XR,ATH,FINANCE
13,BSOFT,831.70,-1.45,-23.67,97.55,50.79,104725.0,-33298.0,107355.0,-2.20,47.0,H-SC,10.68,151.0,-0.32,0.74,25.00,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.60,-15.33,18.13,0.02,26795.0,-26763.0,147796.0,125.45,52.0,M-SC,7.24,234.0,-1.00,1.02,28.56,OX40N,NTT,PAINTS
69,SURYODAY,216.00,1.02,-17.89,49.83,23.03,73272.0,-32028.0,147043.0,60.09,58.0,H-SC,5.36,167.0,-0.44,1.02,45.90,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,3.81,-25.77,52.09,12.90,84489.0,-56298.0,162198.0,139.18,73.0,H-SC,10.96,161.0,-0.67,1.12,34.87,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,11.14,-24.54,85.00,39.61,65748.0,-25150.0,77350.0,-48.92,69.0,H-SC,32.19,135.0,-0.38,0.54,25.20,AR,ATH,MISC
17,COALINDIA,484.83,-0.16,4.94,13.61,19.23,21490.0,7438.0,157898.0,22.81,81.0,L-LC,5.65,179.0,0.35,1.09,27.27,XY25,ATH,MINING
74,TITAGARH,1548.00,-2.76,-19.13,80.21,45.74,157093.0,-46325.0,195852.0,7.85,53.0,H-SC,2.98,174.0,-0.29,1.36,27.38,XY24,NTT,ENGINEERING
82,VOLTAS,1530.00,0.56,16.18,3.02,19.69,6728.0,31023.0,222765.0,4.86,70.0,X-MC,3.59,78.0,4.61,1.54,24.08,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.08
1,25,44.08
2,50,76.43


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.83
MC    30.89
LC    26.25
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.46
X40      23.84
X40N     12.71
XY25     10.71
XR        9.08
AR        8.67
OX40N     7.55
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.38
H-SC    22.95
X-LC    22.80
M-SC    11.26
X-SC     7.84
H-MC     4.69
M-MC     1.46
M-LC     1.28
L-LC     1.09
H-LC     1.08
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.94,-7.68,43.01
IT,12.60,-17.59,76.90
FINANCE,11.15,-10.79,57.28
MISC,7.22,-24.66,75.65
ELECTRICAL,5.95,-9.28,49.30
PAINTS,5.74,-11.62,28.20
INSURANCE,4.73,0.70,33.29
PHARMA,3.99,-2.23,34.40
AUTO,2.82,-28.52,73.08


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3152029.0,21
AR,1321347.0,10
XR,1181510.0,12
X40,1081553.0,15
X40N,906613.0,9
OX40N,700296.0,10
XY25,347020.0,7
SR,283564.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3483638.0,24
X-MC,1465411.0,16
M-SC,1431350.0,15
X-LC,935281.0,13
X-SC,786780.0,8
H-MC,403372.0,3
L-SC,176952.0,2
M-LC,107268.0,1
H-LC,77234.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1230469.0      6
           AR         909102.0      5
M-SC       XY24       818259.0      6
H-SC       XR         756289.0      7
X-MC       X40        475518.0      7
           XY24       435162.0      3
X-LC       X40        416853.0      6
X-MC       X40N       384655.0      4
X-SC       X40N       307788.0      3
M-SC       OX40N      305686.0      5
H-SC       OX40N      304214.0      4
X-SC       XY24       289810.0      3
H-SC       SR         283564.0      2
X-LC       X40N       214170.0      2
H-MC       AR         208499.0      2
           XY24       194873.0      1
X-SC       X40        189182.0      2
X-LC       XY24       183456.0      2
M-SC       XR         180893.0      2
X-MC       XY25       170076.0      2
M-SC       AR         126512.0      2
X-LC       XY25       120802.0      3
M-LC       XR         107268.0      1
L-SC       OX40N       90396.0      1
           XR          86556.0      1
H-LC       AR          77234.0      1
L-MC       XR          50504.0      1
M-MC       XY25        34652.0      1
L-LC       XY25        21490.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
